In [1]:
from scapy.all import rdpcap

In [6]:
filename = "capture_malariaDoS.pcap"

keys = [
  'mqtt.hdrflags', 
  'mqtt.msg', 
  'mqtt.msgid', 
  'mqtt.len', 
  'tcp.flags', 
  'mqtt.kalive', 
  'mqtt.conflag.cleansess', 
  'mqtt.willmsg_len', 
  'mqtt.qos', 
  'tcp.time_delta', 
  'mqtt.dupflag', 
  'mqtt.willtopic', 
  'mqtt.conflags', 
  'mqtt.protoname', 
  'tcp.len', 
  'mqtt.msgtype', 
  'mqtt.proto_len', 
  'mqtt.ver'
]

In [3]:
keys_of_interest = {
    "TCP": ["flags", "time_delta", "len"],
    "MQTT": [
        "hdrflags", "msg", "msgid", "len", "kalive", "conflag.cleansess",
        "willmsg_len", "qos", "dupflag", "willtopic", "conflags", "protoname",
        "msgtype", "proto_len", "ver"
    ]
}

In [4]:
def extract_details(packet):
  details = {}

  # Check for TCP layer
  if "TCP" in packet:
    tcp_layer = packet["TCP"]
    # Handle potential missing attributes using try-except
    details.update({
        key: getattr(tcp_layer, key) if hasattr(tcp_layer, key) else None
        for key in keys_of_interest["TCP"]
    })

  # Check for presence of payload
  if "payload" in packet:
    payload_layer = packet["payload"]

    # Check if payload layer name matches an MQTT layer (optional)
    if hasattr(payload_layer, "name") and payload_layer.name in keys_of_interest:
      details.update({key: getattr(payload_layer, key) for key in keys_of_interest[payload_layer.name]})
    else:
      # Handle unknown payload layers (optional)
      # You might need additional libraries or logic here if your MQTT data
      # isn't in a standard format within the payload.
      pass

  return details


In [7]:
packets = rdpcap(filename)
packets

<capture_malariaDoS.pcap: TCP:130223 UDP:0 ICMP:0 Other:0>

In [8]:
filtered_packets = []
for packet in packets:
  details = extract_details(packet)
  if details:  # Only keep packets with extracted details
    filtered_packets.append(details)
filtered_packets[:5]

[{'flags': <Flag 2 (S)>, 'time_delta': None, 'len': None},
 {'flags': <Flag 2 (S)>, 'time_delta': None, 'len': None},
 {'flags': <Flag 2 (S)>, 'time_delta': None, 'len': None},
 {'flags': <Flag 2 (S)>, 'time_delta': None, 'len': None},
 {'flags': <Flag 2 (S)>, 'time_delta': None, 'len': None}]

In [15]:
packet = packets[23]
packet

<Ether  dst=8c:85:90:b3:e6:d9 src=bc:5f:f4:65:87:30 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=52 id=16522 flags=DF frag=0 ttl=128 proto=tcp chksum=0x3634 src=192.168.1.91 dst=192.168.1.90 |<TCP  sport=1883 dport=54547 seq=3522975880 ack=3010106254 dataofs=8 reserved=0 flags=SA window=65535 chksum=0x9d0d urgptr=0 options=[('MSS', 1460), ('NOP', None), ('WScale', 8), ('NOP', None), ('NOP', None), ('SAckOK', b'')] |>>>

In [12]:
tcp_flags = packet["TCP"].flags
tcp_flags

<Flag 2 (S)>

In [11]:
time_stamp = packet.time
time_stamp

Decimal('1591799819.003272')

In [16]:
tcp_flags = packet["IP"].len
tcp_flags

52